# Noise Model Examples

* *Author*: Christopher J. Wood (cjwood@us.ibm.com)
* *Last Updated*: December 10, 2018

**TODO: add more text**

In [1]:
import numpy as np

# Import Terra
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Import Aer
import qiskit_aer
from qiskit_aer.backends import QasmSimulator

from qiskit_aer import noise
from qiskit_aer.noise import NoiseModel, QuantumError, ReadoutError

## Noise Module

This notebook introduces some of the features of the Qiskit Aer noise module and shows how to  construct a noise model for noisy simulation on the Qasm Simulator

## Errors Module

In [2]:
from qiskit_aer.noise.errors import *
help(noise.errors)

Help on package qiskit_aer.noise.errors in qiskit_aer.noise:

NAME
    qiskit_aer.noise.errors - Standard error module for Qiskit Aer.

DESCRIPTION
    This module contains functions to generate QuantumError objects for
    standard noise channels in quantum information science.
    
    errors (package)
    |
    |-- kraus_error
    |-- mixed_unitary_error
    |-- coherent_unitary_error
    |-- pauli_error
    |-- depolarizing_error
    |-- thermal_relaxation_error
    |-- phase_amplitude_damping_error
    |-- amplitude_damping_error
    |-- phase_damping_error

PACKAGE CONTENTS
    standard_errors

FILE
    /Users/cjwood/anaconda/envs/qiskit-dev/lib/python3.6/site-packages/qiskit_aer/noise/errors/__init__.py




## Test Circuit

In [3]:
# Test circuit
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circ = QuantumCircuit(qr, cr)
circ.h(qr[0])
circ.cx(qr[0], qr[1])
circ.measure(qr, cr)

### Ideal Simulation

In [4]:
# Ideal Backend
backend = QasmSimulator()
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()
print(result.get_counts(circ))

{'00': 532, '11': 468}


### Depolarizing Channel Noise

In [5]:
# Errors
error1 = depolarizing_error(1, 1, standard_gates=True)
print(error1, "\n")
error2 = depolarizing_error(1, 2, standard_gates=True)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, "h")
noise_model.add_quantum_error(error2, "cx", [0, 1])
print(noise_model)
# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()
print("\nCounts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.25, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.25, QobjInstructions = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.25, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.25, QobjInstructions = [[{'name': 'id', 'qubits': [0]}] 

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.0625, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [1]}]
  P(4) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(5) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(6) = 0.0625, QobjInstructions = [[{'name': 'z', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(7) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [1]}]
  P(8) = 0.0625

### X-90 based deplarizing error model

In [6]:
# Errors
error1 = depolarizing_error(1, 1, standard_gates=True)
print(error1, "\n")
error2 = depolarizing_error(1, 2, standard_gates=True)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.set_x90_single_qubit_gates(['h'])
noise_model.add_all_qubit_quantum_error(error1, "x90")
noise_model.add_quantum_error(error2, "cx", [0, 1])

print(noise_model)
# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()
print("\nCounts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.25, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.25, QobjInstructions = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.25, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.25, QobjInstructions = [[{'name': 'id', 'qubits': [0]}] 

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.0625, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [1]}]
  P(4) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(5) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(6) = 0.0625, QobjInstructions = [[{'name': 'z', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(7) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [1]}]
  P(8) = 0.0625

### Depolarizing noise from Kraus matrices

In [7]:
# We define a unitary channel as a Kraus set
# Note that internally this will be converted to a mixed unitary error channel

I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Errors
kraus_ops1 = [np.sqrt(0.25) * I, np.sqrt(0.25) * X, np.sqrt(0.25) * Y, np.sqrt(0.25) * Z]
error1 = kraus_error(kraus_ops1)
print(error1, "\n")

kraus_ops2 = [np.kron(a,b) for a in kraus_ops1 for b in kraus_ops1]
error2 = kraus_error(kraus_ops2)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, "h")
noise_model.add_all_qubit_quantum_error(error2, "cx")
print(noise_model, "\n")

backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()

print("Counts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.25, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.25, QobjInstructions = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.25, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.25, QobjInstructions = [[{'name': 'id', 'qubits': [0]}] 

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.0625, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [1]}]
  P(4) = 0.0625, QobjInstructions = [[{'name': 'x', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(5) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(6) = 0.0625, QobjInstructions = [[{'name': 'z', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(7) = 0.0625, QobjInstructions = [[{'name': 'y', 'qubits': [1]}]
  P(8) = 0.0625

### Classical readout error

In [8]:
# Error
p_given_0 = [0.9, 0.1]
p_given_1 = [0.3, 0.7]
error = ReadoutError([p_given_0, p_given_1])
print(error, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_readout_error(error)
print(noise_model, "\n")

# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()
print("Counts:")
print(result.get_counts(circ))

ReadoutError on 1 qubits. Assignment probabilities:
 P(j|0) =  [0.9, 0.1]
 P(j|1) =  [0.3, 0.7] 

NoiseModel:
  Instructions with noise: ['measure']
  All-qubits errors: ['measure'] 

Counts:
{'01': 137, '00': 488, '10': 142, '11': 233}


### Amplitude Damping Error

In [9]:
# Error
error1 = amplitude_damping_error(0.5)
print(error1, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, "h")
print(noise_model, "\n")

# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()

# Results
print("Counts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 1.0, QobjInstructions = [[{'name': 'kraus', 'qubits': [0], 'params': [array([[ 1.        +0.j, -0.        +0.j],
       [-0.        +0.j,  0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.        +0.j, 0.        +0.j]])]}] 

NoiseModel:
  Instructions with noise: ['h']
  All-qubits errors: ['h'] 

Counts:
{'00': 761, '11': 239}


### Phase damping error

In [10]:
error1 = phase_damping_error(0.25)
print(error1, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, "h")
print(noise_model, "\n")

# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()

# Counts
print("Counts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.066987298108, QobjInstructions = [[{'name': 'z', 'qubits': [0]}]
  P(1) = 0.933012701892, QobjInstructions = [[{'name': 'id', 'qubits': [0]}] 

NoiseModel:
  Instructions with noise: ['h']
  All-qubits errors: ['h'] 

Counts:
{'00': 492, '11': 508}


### Pauli Error (matrix)

In [11]:
# Errors
error2 = pauli_error([('II', 0.5), ('IX', 0.5)], standard_gates=True)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_quantum_error(error2, "cx", [0, 1])
print(noise_model)
# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()
print("\nCounts:")
print(result.get_counts(circ))

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.5, QobjInstructions = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.5, QobjInstructions = [[{'name': 'id', 'qubits': [0]}] 

NoiseModel:
  Instructions with noise: ['cx']
  Specific qubit errors: [('cx', [0, 1])]

Counts:
{'01': 293, '00': 231, '10': 232, '11': 244}


### Pauli Error (gates)

In [12]:
# Errors
error2 = pauli_error([('II', 0.5), ('ZX', 0.5)], standard_gates=False)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_quantum_error(error2, "cx", [0, 1])
print(noise_model)
# Backend
backend = QasmSimulator()
qobj = qiskit.compile(circ, backend, shots=1000,
                      basis_gates=noise_model.basis_gates)
job = backend.run(qobj, noise_model=noise_model)
result = job.result()
print("\nCounts:")
print(result.get_counts(circ))

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.5, QobjInstructions = [[{'name': 'unitary', 'qubits': [0, 1], 'params': array([[ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j, -0.+0.j, -1.+0.j],
       [ 0.+0.j,  0.+0.j, -1.+0.j, -0.+0.j]])}]
  P(1) = 0.5, QobjInstructions = [[{'name': 'id', 'qubits': [0]}] 

NoiseModel:
  Instructions with noise: ['cx']
  Specific qubit errors: [('cx', [0, 1])]

Counts:
{'01': 275, '00': 220, '10': 241, '11': 264}
